In [ ]:
def two_opt(adjacency_matrix, route):
    """Performs the 2-opt algorithm to improve a given route."""

    def calculate_distance(city1, city2):
        return adjacency_matrix[city1][city2]

    improvement = True
    while improvement:
        improvement = False
        for i in range(len(route) - 2):
            for j in range(i + 2, len(route)):
                if i == 0 and j == len(route) - 1:
                    continue  # Skip swapping start and end points

                current_distance = calculate_distance(route[i], route[i + 1]) + calculate_distance(route[j], route[(j + 1) % len(route)])
                new_distance = calculate_distance(route[i], route[j]) + calculate_distance(route[i + 1], route[(j + 1) % len(route)])

                if new_distance < current_distance:
                    route[i + 1 : j + 1] = route[j : i : -1]  # Reverse the sub-route
                    improvement = True
    return route





def christofides_partiel(matrice_distances, solution_partielle):
    G = nx.complete_graph(len(matrice_distances))
    for u, v in G.edges():
        G[u][v]['weight'] = matrice_distances[u][v]

    T = minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)

    matching = nx.algorithms.matching.max_weight_matching(odd_graph)
    for u, v in matching:
        T.add_edge(u, v, weight=matrice_distances[u][v])

    if nx.is_eulerian(T):
        eulerian_circuit = list(nx.eulerian_circuit(T))
    else:
        raise Exception("Le graphe résultant n'est pas eulérien")

    visited = set(solution_partielle)
    next_city = None
    for u, v in eulerian_circuit:
        if u not in visited:
            next_city = u
            break
        if v not in visited:
            next_city = v
            break

    if next_city is not None:
        solution_partielle.append(next_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def nearest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    nearest_city = min(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(nearest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def loneliest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    loneliest_city = max(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(loneliest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

In [ ]:
import numpy as np
import random
import math

def simulated_annealing(cities,current_path, initial_temperature=100000, cooling_rate=0.88, nb_iter=50000, cooldown_interval=50, cooldown_factor=1.1):
    def total_distance(path, cities):
      distance = 0
      for i in range(len(path) - 1):
          distance += cities[path[i], path[i + 1]]
      distance += cities[path[-1], path[0]]  # Retour à la première ville

      return distance
    #Generation du voisin
    def generate_neighbor(path):
        i, j = sorted(random.sample(range(len(path)), 2)) #Choisir 2 villes aleatoire
        return path[:i] + path[j:j+1] + path[i+1:j] + path[i:i+1] + path[j+1:] #Permuter entre ces 2 villes

    #initialisation
    print("Path initial : " , current_path)
    current_distance = total_distance(current_path, cities)
    print("Sa distance : " , current_distance)
    best_path = current_path.copy()
    best_distance = current_distance
    temperature = initial_temperature
    no_improvement_counter = 0


    for _ in range(nb_iter):
        new_path = generate_neighbor(current_path) #generer un voisin
        new_distance = total_distance(new_path, cities)
        delta_distance = new_distance - current_distance
        temperature=max(0.0002,temperature)
        if delta_distance < 0 or random.random() < math.exp(min(0, -delta_distance / temperature)):
          #On prend le voisin si il ameliore la fonction objectif ou avec une certain probabilite
            current_path = new_path
            current_distance = new_distance


            if current_distance < best_distance:
              #Si le voisin ameliore, il devient la meilleure solution
                best_path = current_path
                best_distance = current_distance

            temperature *= cooling_rate #demunition de temperature
            no_improvement_counter = 0

        else:
          #Si on prend pas le voisin, augmenter le compteur
            no_improvement_counter += 1

        if no_improvement_counter >= cooldown_interval:
          #Si aucune solution n’est acceptée après cooldown_interval, incrémenter la valeur de la température
            temperature *= cooldown_factor
            no_improvement_counter = 0 #reinitialiser le compteur

    return best_path, best_distance

